# Block Model Reports

This notebook demonstrates how to create and run resource estimation reports on block models.

Reports provide tonnage, grade, and metal content summaries grouped by categories (e.g., geological domains).

## Requirements for Reports

1. **Units on columns** - Report columns must have units defined (e.g., `g/t` for grades)
2. **At least one category column** - For grouping results (e.g., domain, rock type)
3. **Density information** - Either a density column or a fixed density value

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

In [ ]:
%load_ext evo.widgets

## Create a Block Model with Sample Data

First, let's create a block model with some grade data that we can report on.

In [ ]:
import numpy as np
import pandas as pd

from evo.blockmodels import Units
from evo.objects.typed import BlockModel, Point3, RegularBlockModelData, Size3d, Size3i

# Define block model geometry
origin = (0, 0, 0)
n_blocks = (10, 10, 5)  # 500 blocks total
block_size = (25.0, 25.0, 10.0)  # metres
total_blocks = n_blocks[0] * n_blocks[1] * n_blocks[2]

# Generate block centroid coordinates
np.random.seed(42)
centroids = []
for k in range(n_blocks[2]):
    for j in range(n_blocks[1]):
        for i in range(n_blocks[0]):
            x = origin[0] + (i + 0.5) * block_size[0]
            y = origin[1] + (j + 0.5) * block_size[1]
            z = origin[2] + (k + 0.5) * block_size[2]
            centroids.append((x, y, z))

# Create sample data with grades and density
block_data = pd.DataFrame(
    {
        "x": [c[0] for c in centroids],
        "y": [c[1] for c in centroids],
        "z": [c[2] for c in centroids],
        "Au": np.random.lognormal(mean=0.5, sigma=0.8, size=total_blocks),  # Gold grade
        "density": np.random.uniform(2.5, 2.9, size=total_blocks),  # Bulk density
    }
)

print(f"Created {len(block_data)} blocks")
print(f"Au grade range: {block_data['Au'].min():.2f} - {block_data['Au'].max():.2f}")
block_data.head()

In [ ]:
import uuid

# Create the block model
bm_data = RegularBlockModelData(
    name=f"Report Demo Block Model - {uuid.uuid4().hex[:8]}",
    description="Block model for demonstrating reports",
    origin=Point3(x=origin[0], y=origin[1], z=origin[2]),
    n_blocks=Size3i(nx=n_blocks[0], ny=n_blocks[1], nz=n_blocks[2]),
    block_size=Size3d(dx=block_size[0], dy=block_size[1], dz=block_size[2]),
    cell_data=block_data,
    crs="EPSG:28354",
    size_unit_id=Units.METRES,
    units={"Au": Units.GRAMS_PER_TONNE, "density": Units.TONNES_PER_CUBIC_METRE},
)

block_model = await BlockModel.create_regular(manager, bm_data)
print(f"Created block model: {block_model.name}")

In [ ]:
# Pretty-print the block model (shows Portal/Viewer links)
block_model

## Add a Domain Column

Reports require at least one category column for grouping. Let's add a simple domain column by slicing the block model into three geological domains based on elevation (z-coordinate).

In practice, domains would come from geological interpretation, but this demonstrates the concept.

In [ ]:
# Get the current block data
df = await block_model.to_dataframe()

# Create domain column based on z-coordinate (elevation)
# Divide into 3 domains: LMS1 (lower), LMS2 (middle), LMS3 (upper)
z_min, z_max = df["z"].min(), df["z"].max()
z_range = z_max - z_min


def assign_domain(z):
    if z < z_min + z_range / 3:
        return "LMS1"  # Lower domain
    elif z < z_min + 2 * z_range / 3:
        return "LMS2"  # Middle domain
    else:
        return "LMS3"  # Upper domain


df["domain"] = df["z"].apply(assign_domain)

# Check domain distribution
print("Domain distribution:")
print(df["domain"].value_counts())

In [ ]:
# Add the domain column to the block model
# Include geometry columns (x, y, z) for block identification
domain_data = df[["x", "y", "z", "domain"]]

version = await block_model.add_attribute(domain_data, "domain")
print(f"Added domain column. New version: {version.version_id}")

In [ ]:
# Refresh to see the new attribute
block_model = await block_model.refresh()
block_model.attributes

## Create a Report

Now we can create a report specification. The report will calculate:
- Tonnage for each domain
- Average Au grade (weighted by mass)
- Total Au metal content

**Key classes for reports:**
- `Aggregation` - Enum: `MASS_AVERAGE` (for grades), `SUM` (for metal content)
- `Units` - Constants for output units (e.g., `Units.GRAMS_PER_TONNE`)
- `MassUnits` - Constants for mass output (e.g., `MassUnits.TONNES`)

In [ ]:
from evo.blockmodels import Units
from evo.blockmodels.typed import (
    Aggregation,
    MassUnits,
    ReportCategorySpec,
    ReportColumnSpec,
    ReportSpecificationData,
)

# Define the report
report_data = ReportSpecificationData(
    name="Gold Resource Report",
    description="Resource estimate by domain for Au",
    columns=[
        ReportColumnSpec(
            column_name="Au",
            aggregation=Aggregation.MASS_AVERAGE,  # Use MASS_AVERAGE for grades
            label="Au Grade",
            output_unit_id=Units.GRAMS_PER_TONNE,  # Use Units class for discoverability
        ),
    ],
    categories=[
        ReportCategorySpec(
            column_name="domain",
            label="Domain",
            values=["LMS1", "LMS2", "LMS3"],  # Optional: specify values to include
        ),
    ],
    mass_unit_id=MassUnits.TONNES,  # Use MassUnits class
    density_column_name="density",  # Use density column for mass calculation
    run_now=True,  # Run immediately after creation
)

# Create the report
report = await block_model.create_report(report_data)
print(f"Created report: {report.name}")

In [ ]:
# Pretty-print the report (shows BlockSync link)
report

## View Report Results

Since we set `run_now=True`, the report was executed automatically. Let's get the results.

In [ ]:
# Get the latest report result (waits if report is still running)
result = await report.refresh()

# Pretty-print the result (displays table in Jupyter)
result

## Create a Report with Fixed Density

If you don't have a density column, you can use a fixed density value instead.

In [ ]:
# Create a report with fixed density
fixed_density_report_data = ReportSpecificationData(
    name="Au Report (Fixed Density)",
    description="Resource estimate using fixed density of 2.7 t/m3",
    columns=[
        ReportColumnSpec(
            column_name="Au",
            aggregation=Aggregation.MASS_AVERAGE,
            label="Au Grade",
            output_unit_id=Units.GRAMS_PER_TONNE,
        ),
    ],
    categories=[
        ReportCategorySpec(column_name="domain", label="Domain"),
    ],
    mass_unit_id=MassUnits.TONNES,
    density_value=2.7,  # Fixed density value
    density_unit_id=Units.TONNES_PER_CUBIC_METRE,  # Unit for fixed density
    run_now=True,
)

fixed_report = await block_model.create_report(fixed_density_report_data)
print(f"Created report: {fixed_report.name}")

## Create a Report with Cut-offs

Reports can also evaluate different cut-off grades. This is useful for grade-tonnage analysis.

In [ ]:
# Create a report with cut-off values
cutoff_report_data = ReportSpecificationData(
    name="Au Grade-Tonnage Report",
    description="Grade-tonnage analysis with Au cut-offs",
    columns=[
        ReportColumnSpec(
            column_name="Au",
            aggregation=Aggregation.MASS_AVERAGE,
            label="Au Grade",
            output_unit_id=Units.GRAMS_PER_TONNE,
        ),
    ],
    categories=[
        ReportCategorySpec(column_name="domain", label="Domain"),
    ],
    mass_unit_id=MassUnits.TONNES,
    density_column_name="density",
    cutoff_column_name="Au",  # Apply cut-off on Au grade
    cutoff_values=[0.5, 1.0, 2.0, 3.0],  # Cut-off values in g/t
    run_now=True,
)

cutoff_report = await block_model.create_report(cutoff_report_data)
print(f"Created report: {cutoff_report.name}")

In [ ]:
# Get and display results
cutoff_result = await cutoff_report.refresh()
cutoff_result

## List All Reports

You can list all report specifications on a block model.

In [ ]:
# List all reports on this block model
reports = await block_model.list_reports()

print(f"Found {len(reports)} report(s):")
for r in reports:
    print(f"  - {r.name} (revision {r.revision})")

## Summary

This notebook demonstrated:

1. **Creating a block model** with grade and density data
2. **Adding a domain column** for grouping (simulating geological domains)
3. **Creating reports** with the typed `Report` API
4. **Viewing results** as DataFrames
5. **Using fixed density** vs density column
6. **Grade-tonnage analysis** with cut-off values
7. **Pretty printing** reports with BlockSync links

The `report` object provides a `blocksync_url` property that links directly to the report in BlockSync for interactive viewing and analysis.